### WebCrawing
- 웹페이지의 종류
    - 정적페이지 : 페이지의 데이터가 변경될 때 URL이 변경 : Html
    - 동적페이지 : 페이지의 데이터가 변경될 때 URL이 변경 X : Json
- requests package
    - 브라우져의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL입력 > Data 가져옴
    - requests 패키지 : URL > Data
    

### Naver Stock Data
- Kospi 주가 지수
- Kosdaq 주가 지수
- USD : 원달러 환율 


### KOSPI 데이터 수집 코드 작성

In [1]:
# 필요한 패키지 import
import requests
import pandas as pd

In [2]:
# 1. ** 웹서비스를 분석 ** : 크롬 개발자도구 사용(트래픽 확인) > URL 얻음
# https://m.stock.naver.com/

In [3]:
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=4"

In [4]:
# 2. requests > response : JSON(str)

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
response.text[:200]

'[{"localTradedAt":"2022-06-22","closePrice":"2,342.81","compareToPreviousClosePrice":"-66.12","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-2.74","openPrice"'

In [7]:
# 3. JSON(str) > list, dict > DataFrame 출력

In [8]:
data = response.json()
type(data), data[:1]

(list,
 [{'localTradedAt': '2022-06-22',
   'closePrice': '2,342.81',
   'compareToPreviousClosePrice': '-66.12',
   'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
   'fluctuationsRatio': '-2.74',
   'openPrice': '2,417.11',
   'highPrice': '2,418.05',
   'lowPrice': '2,342.81'}])

In [9]:
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-10,"2,595.87"
9,2022-06-09,"2,625.44"


### KOSDAQ 데이터 수집 코드 작성

In [10]:
# 1. 웹서비스를 분석

In [11]:
url = "https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=4"

In [12]:
# 2. requests > response : JSON(str)

In [13]:
response = requests.get(url)
response.text[:200]

'[{"localTradedAt":"2022-06-22","closePrice":"746.96","compareToPreviousClosePrice":"-31.34","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-4.03","openPrice":"'

In [14]:
# 3. JSON(str) > list, dict > DataFrame 출력

In [15]:
data = response.json()
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
df.tail(2)

,localTradedAt,closePrice
8,2022-06-10,869.86
9,2022-06-09,877.18


### 함수 만들기
- param : pagesize, page

In [22]:
def stock_price(pagesize, page, code='KOSPI'):
    """ This function is crawling stock price from naver webpage
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns
    
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    
    response = requests.get(url)
    data = response.json()
    
    return pd.DataFrame(data)[["localTradedAt", "closePrice"]]

In [19]:
# 최근 60일치 데이터 가져오기
kospi = stock_price(60, 1, 'KOSPI')
kosdaq = stock_price(60, 1, 'KOSDAQ')

In [21]:
# docstring : 함수를 사용하는 방법을 문자열로 작성
# help(), shift + tab 이용
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling stock price from naver webpage
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns



### 원달러 환율 데이터 수집
- 시장지표 > 미국 > 일별시세(더보기)

In [ ]:
# 최근 60일치 원달러 환율 데이터 수집

In [34]:
# 1. 웹페이지 분석
page, pagesize = 1, 60
url = f"https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page={page}&pageSize={pagesize}"

In [35]:
# 2. requests > response : JSON(str)
response = requests.get(url)
response.text[:200]

'[{"localTradedAt":"2022-08-03","closePrice":"1,309.70","fluctuations":"-2.30","fluctuationsRatio":"-0.18","fluctuationsType":{"code":"5","text":"하락","name":"FALLING"},"cashBuyValue":"1,332.61","cashSe'

In [36]:
# 3. JSON(str) > list, dict > DataFrame 출력

In [37]:
data = response.json()

In [39]:
usd = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


In [41]:
def usd_price(pagesize, page, code="USD"):
    """ This function is crawling usd price from naver webpage
    
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : nation code
    
    Return
    ------
    type : DataFrame : display dataframe
    
    """
    url = f"https://api.stock.naver.com/marketindex/exchange/FX_{code}KRW/prices?page={page}&pageSize={pagesize}"
    
    response = requests.get(url)
    data = response.json()
    
    return pd.DataFrame(data)[["localTradedAt", "closePrice"]]

In [44]:
usd_price(10, 1, code="EUR")

,localTradedAt,closePrice
0,2022-08-03,"1,333.64"
1,2022-08-02,"1,337.98"
2,2022-08-01,"1,339.32"
3,2022-07-29,"1,333.20"
4,2022-07-28,"1,323.87"
5,2022-07-27,"1,329.24"
6,2022-07-26,"1,329.35"
7,2022-07-25,"1,339.95"
8,2022-07-22,"1,340.39"
9,2022-07-21,"1,339.06"


In [ ]:
# 데이터 분석
# 상관관계분석 : 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석방법
# 원달러 환율이 높으면 코스피, 코스닥 지수가 낮다.

In [ ]:
# 피어슨 상관계수
# 1과 가까울수록 강한 양의 상관관계를 갖는다.
# -1과 가까울수록 강한 음의 상관관계를 갖는다.
# 0과 가까울수록 관계가 없다.

In [49]:
df = kospi.copy()
df["kosdaq"] = kosdaq['closePrice']
df['usd'] = usd['closePrice']
df = df.rename(columns={'closePrice' : 'kospi'})
df.head()

,localTradedAt,kospi,kosdaq,usd
0,2022-08-03,"2,456.12",813.72,"1,309.70"
1,2022-08-02,"2,439.62",804.34,"1,312.00"
2,2022-08-01,"2,452.25",807.61,"1,305.00"
3,2022-07-29,"2,451.50",803.62,"1,304.50"
4,2022-07-28,"2,435.27",798.32,"1,302.00"


In [59]:
# 컬럼의 데이터 타입 변경 : str > float
# df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력
# lamda
df['kospi'] = df['kospi'].apply(lambda data: float(data.replace(',', '')))
df['kosdaq'] = df['kosdaq'].apply(lambda data: float(data.replace(',', '')))
df['usd'] = df['usd'].apply(lambda data: float(data.replace(',', '')))

In [62]:
df.dtypes

localTradedAt     object
kospi            float64
kosdaq           float64
usd              float64
dtype: object

In [63]:
df[['kospi', 'kosdaq', 'usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984042,-0.878291
kosdaq,0.984042,1.000000,-0.821246
usd,-0.878291,-0.821246,1.000000


In [ ]:
# kospi - kosdaq : 0.984 -> 1과 가까우면 강한 양의 상관관계
# kospi - usd : -0.878 -> -1과 가까우면 강함 음의 상관관계
# 현재 원달러 환율이 높으니 kospi가 낮으니 지금 코스피 주식을 사자

In [ ]:
# copy()

In [69]:
data1 = [1, 2, 3]

data2 = data1 # 얕은 복사: 주소 복사, 같은 주소값을 사용한다.
data3 = data1.copy() # 깊은 복사: 값복사, 새로운 주소값을 만든다.

print(data1, data2) # [1, 2, 3] [1, 2, 3]

data1[1] = 4
print(data1, data2) # [1, 4, 3] [1, 4, 3]
print(data1, data2, data3) # [1, 4, 3] [1, 4, 3] [1, 2, 3]

[1, 2, 3] [1, 2, 3]
[1, 4, 3] [1, 4, 3]
[1, 4, 3] [1, 4, 3] [1, 2, 3]


In [ ]:
# apply(func) : 모든 데이터를 func을 적용시킨 결과 출력

In [71]:
df = pd.DataFrame([{"age" : 23}, {"age" : 36}, {"age" : 27}])
df

,age
0,23
1,36
2,27


In [72]:
# 연령대 컬럼 추가
def change_ages(age):
    return age//10 * 10

In [74]:
df['ages'] = df['age'].apply(change_ages)
df

,age,ages
0,23,20
1,36,30
2,27,20


In [ ]:
# lambda : 일회성 함수
# 사용이유 : 간단한 함수(파라미터를 받아서 바로 리턴하는 함수)를 메모리 절약하여 사용

In [76]:
# 함수 3개 사용, 메모리 3칸 사용
def plus(n1, n2):
    return n1 + n2

def minus(n1, n2):
    return n1 - n2

def calc(func, n1, n2):
    return func(n1, n2)

In [77]:
calc(plus, 1, 2), calc(minus, 1, 2)

(3, -1)

In [79]:
plus_lambda = lambda n1, n2: n1+n2
plus(1, 2), plus_lambda(1, 2)

(3, 3)

In [78]:
# 함수 1개 사용, 메모리 1개 사용
def calc(func, n1, n2):
    return func(n1, n2)

calc(lambda n1, n2: n1+n2, 1, 2), calc(lambda n1, n2: n1-n2, 1, 2)

(3, -1)